In [1]:
import pandas as pd
import base64
import numpy as np

In [2]:
import urllib
import nltk
import sys, os
from typing import List
import base64
from sqlalchemy import create_engine
import iso639
import psycopg2 as pg
pd.set_option('display.max_colwidth', 1)

# Connecting to the db

In [3]:
def getCursor():
    database_settings = "dbname=" + dbname + " user=" + user + " host=" + host + " port=" + port + " password=" + password

    engine_url = 'postgresql://' + user + ':' + password + '@' + host + ':'+ port +'/'+dbname
    engine = create_engine(engine_url)

    conn = pg.connect(database_settings)
    cur = conn.cursor()
    return cur,conn,engine

# Encrypting postgres password

In [4]:
#password = 'insert password'.encode("utf-8")
#encoded = base64.b64encode(password)

#print(encoded)

In [5]:
dbname = 'socialmedia'
user= 'helen'
host= 'humandata.zgis.at'
port= '5433'
file = open('humandata_key.key', 'rb')
key = file.read() # The key will be type bytes
file.close()
password = base64.b64decode(key).decode("utf-8")
cur,conn,engine = getCursor()

In [6]:
schema = 'movement_detection'
table = 'data_2019_final_selection'
schematable = schema + '.' + table

In [9]:
#schematable = "public.test_sample_location_extraction"
schematable = "movement_detection.data_2019_final_selection"
SQL = "SELECT * FROM " + schematable  
print(SQL)
data_plus = pd.read_sql_query(SQL,con=conn)

SELECT * FROM movement_detection.data_2019_final_selection


In [10]:
data_plus.head(2)

,message_id,date,text,source,geom,longitude,latittude
0,1181425472770494464,2019-10-08 04:25:57,@WinTheDay848 😍,"<a href=""http://tapbots.com/tweetbot"" rel=""nofollow"">Tweetbot for iΟS</a>",0101000020E610000096CDDA75C89F5EC083F7FAFF5DB64640,-122.49661,45.424744
1,1187179824189788160,2019-10-24 01:31:41,Also pastors,"<a href=""http://tapbots.com/tweetbot"" rel=""nofollow"">Tweetbot for iΟS</a>",0101000020E610000077851D4A9DE655C0B64BFAFFE1E64440,-87.60335,41.803772


In [11]:
data_plus['source'].unique()

array(['<a href="http://tapbots.com/tweetbot" rel="nofollow">Tweetbot for iΟS</a>',
       '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
       '<a href="http://instagram.com" rel="nofollow">Instagram</a>',
       '<a href="http://foursquare.com" rel="nofollow">Foursquare</a>',
       '<a href="https://www.hootsuite.com" rel="nofollow">Hootsuite Inc.</a>',
       '<a href="https://www.swarmapp.com" rel="nofollow">Foursquare Swarm</a>',
       '<a href="https://untappd.com" rel="nofollow">Untappd</a>'],
      dtype=object)

In [12]:
data_plus = data_plus.replace({'<a href="http://tapbots.com/tweetbot" rel="nofollow">Tweetbot for iΟS</a>':'ios',
                              '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>':'iphone',
                              '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>':'Android',
                              '<a href="http://instagram.com" rel="nofollow">Instagram</a>':'Instagram',
                              '<a href="http://foursquare.com" rel="nofollow">Foursquare</a>':'Foursquare',
                              '<a href="https://www.hootsuite.com" rel="nofollow">Hootsuite Inc.</a>':'Hootsuite',
                              '<a href="https://www.swarmapp.com" rel="nofollow">Foursquare Swarm</a>':'fswarm',
                              '<a href="https://untappd.com" rel="nofollow">Untappd</a>':'Untapped' })

In [13]:
data_plus['source'].unique()

array(['ios', 'iphone', 'Android', 'Instagram', 'Foursquare', 'Hootsuite',
       'fswarm', 'Untapped'], dtype=object)

In [14]:
data_plus.shape

(164599, 7)

In [16]:
data_plus.columns

Index(['message_id', 'date', 'text', 'source', 'geom', 'longitude',
       'latittude'],
      dtype='object')

In [17]:
data_plus = data_plus[['message_id', 'date', 'text', 'source','longitude',
       'latittude']]

data_plus.head(3)

,message_id,date,text,source,longitude,latittude
0,1181425472770494464,2019-10-08 04:25:57,@WinTheDay848 😍,ios,-122.496610,45.424744
1,1187179824189788160,2019-10-24 01:31:41,Also pastors,ios,-87.603350,41.803772
2,1161754851510030342,2019-08-14 21:41:55,we are in and the tickets are REAL https://t.co/xkWBsk9jzE,iphone,-96.712301,40.817276


In [18]:
data_plus.to_csv('data2/data2_complete.csv')

# Join datasets

Joining the datasets extracted from multiple sources

In [ ]:
df = pd.read_csv('data//main3.csv')
df1 = pd.read_csv('data//fours.csv')
df2 = pd.read_csv('data//fswarm.csv')
df3 = pd.read_csv('data//insta.csv')
df4 = pd.read_csv('data//hootsuite.csv')
df5 = pd.read_csv('data//untapped.csv')

In [ ]:
print(df.shape, df1.shape, df2.shape, df3.shape, df4.shape, df5.shape)

In [ ]:
frames = [df, df1, df2, df3, df4, df5]

In [ ]:
df_final = pd.concat(frames)
df_final[['text']]

In [ ]:
df_final.columns

In [ ]:
df_final =df_final.drop(columns = ['message_id', 'date', 'source', 'geom', 'user_id',
       'user_name'])

In [ ]:
df_final.columns

In [ ]:
df_final.to_csv('data//data.csv')

# Join location datasets

In [2]:
df = pd.read_csv('data//data_english_locations.csv')
df2 = pd.read_csv('data//data_non_english_locations1.csv')
df3 = pd.read_csv('data//data_non_english_locations2.csv')
df4 = pd.read_csv('data//data_non_english_locations3.csv')
print (df.shape, df2.shape, df3.shape, df4.shape)

(67804, 12) (4996, 13) (3649, 13) (3000, 12)


In [3]:
df.columns

Index(['Unnamed: 0', 'id', 'date', 'text', 'source', 'longitude', 'latittude',
       'lang', 'GPE', 'FAC', 'LOC', 'ORG'],
      dtype='object')

In [4]:
df = df.rename(columns ={'text':'en_text'})
df2 = df2.rename(columns ={'de_text':'en_text'})
df3 =df3.rename(columns ={'de_text':'en_text'})
df4 =df4.rename(columns ={'de_text':'en_text'})
df.tail()

,Unnamed: 0,id,date,en_text,source,longitude,latittude,lang,GPE,FAC,LOC,ORG
67799,67799,6590098,2020-02-17 19:01:22,Only the third batch ever - Drinking a Propiti...,"<a href=""https://untappd.com"" rel=""nofollow"">U...",-122.8000,38.5319,en,[],[],"[Russian River Brewing Company, Russian River ...",[]
67800,67800,1492661,2020-03-10 23:37:46,@loonballoonfun! - Drinking a Tropical IPA by ...,"<a href=""https://untappd.com"" rel=""nofollow"">U...",-81.2308,33.9952,en,[],[],"[Steel Hands Brewing, Carolina Wings & Rib House]",[]
67801,67801,6363492,2020-02-15 19:24:04,"Dave & Busters serving the coldest, freshest M...","<a href=""https://untappd.com"" rel=""nofollow"">U...",-88.0484,43.0581,en,[],[],[Dave & Busters],[]
67802,67802,954252,2020-02-23 20:10:48,Drinking a Devil Dancer (2019) by @foundersbre...,"<a href=""https://untappd.com"" rel=""nofollow"">U...",-84.6607,42.7413,en,[],[Horrocks Tavern],[],[]
67803,67803,3089718,2020-01-20 04:40:55,Drinking a Straight C+s IPA by @SpiderBiteBeer...,"<a href=""https://untappd.com"" rel=""nofollow"">U...",-75.4587,41.1107,en,[],[],[],[]


In [5]:
df4.columns

Index(['Unnamed: 0', 'id', 'date', 'source', 'longitude', 'latittude', 'lang',
       'en_text', 'GPE', 'FAC', 'LOC', 'ORG'],
      dtype='object')

In [6]:
df = df.drop(columns =['Unnamed: 0'])
df2 = df2.drop(columns =['Unnamed: 0','text'])
df3 = df3.drop(columns =['Unnamed: 0','text'])
df4 = df4.drop(columns =['Unnamed: 0'])
df3.head()

,id,date,source,longitude,latittude,lang,en_text,GPE,FAC,LOC,ORG
0,6731721,2020-02-19 01:40:39,"<a href=""http://twitter.com/download/android"" ...",-100.199042,25.641778,es,@CicGPE @Telmex @Telmexsolizes Alert very low ...,"[Rincon de la Sierra, Guadeloupe]",[],[],[]
1,6732978,2020-02-19 01:56:57,"<a href=""http://twitter.com/download/iphone"" r...",-76.914535,38.964903,pt,Responding firm research and happy with the re...,[],[],[Geneo de Todexs],[]
2,6733425,2020-02-19 02:05:01,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",-88.049426,43.022125,nl,@BadgerMBB @aleemty2 Nice,[],[],[],[]
3,6733689,2020-02-19 02:10:12,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",-74.067041,40.724030,af,Took me OUT,[],[],[],[]
4,6734107,2020-01-08 07:11:06,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",-122.038311,47.531555,af,"@cjbrinton Sleep boo boo, sleep",[],[],[],[]


In [13]:
frames = [df2,df3,df4]

In [14]:
df_final = pd.concat(frames)
df_final.shape

(11645, 11)

In [15]:
df_final = df_final.drop_duplicates(subset =['en_text','id','latittude'], keep = 'last')
df_final.shape

(10189, 11)

In [17]:
df_final.to_csv('data//complete_4_locations_transla.csv')

In [12]:
df.head()

,id,date,en_text,source,longitude,latittude,lang,GPE,FAC,LOC,ORG
0,3242090,2020-02-04 08:52:49,@amandarykoff Why am I not surprised,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",-122.282159,37.839600,en,[],[],[],[]
1,3242118,2020-02-04 08:53:34,Whoever wins Iowa isnt even going to get a goo...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",-122.281984,37.839508,en,"[Iowa, Iowa]",[],[],[]
2,3242119,2020-02-04 08:53:42,Im going to bed. Whats the point of staying awake,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",-122.281984,37.839508,en,[],[],[],[]
3,3242369,2020-04-17 08:52:16,@JimOBrienWX good good morning my brother I'm ...,"<a href=""http://twitter.com/download/android"" ...",-85.684524,40.106547,en,[],[],[],[]
4,3242397,2020-03-26 02:03:58,Seeing lots of people walking running around t...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",-121.963962,37.951202,en,[],[],[],[]


In [19]:
df_temp1 = pd.read_csv('data//locations_temp_information_full.csv')
df_temp = pd.read_csv('data//locations_temp_information_full_trans.csv')


In [20]:
df_temp1.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'date', 'en_text', 'source',
       'longitude', 'latittude', 'lang', 'GPE', 'FAC', 'LOC', 'ORG', 'TE'],
      dtype='object')

In [21]:
df_temp.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'date', 'source', 'longitude',
       'latittude', 'lang', 'en_text', 'GPE', 'FAC', 'LOC', 'ORG', 'TE'],
      dtype='object')

In [22]:
df_temp1.shape

(67804, 14)

In [23]:
df_temp.shape

(10189, 14)

In [24]:
frame = [df_temp1, df_temp]

In [25]:
temp = pd.concat(frame)

In [26]:
temp.shape

(77993, 14)

In [27]:
temp.to_csv('data//analysis_data//full_entity_extraction.csv')